# Privileged Role Assignment Outside Change Window

**Privileged role/group changes outside approved hours may indicate compromise or policy bypass.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **actor identity**, **approval/ticket**, **PIM events**, **source IP/device**.


## Investigation Queries

Find role assignment events and check who initiated them and from where.


In [ ]:
// KQL
AzureActivity
| where TimeGenerated > ago(30d)
| where OperationNameValue has_any ('RoleAssignmentCreated','Add member to role','Add user to group')
| where Caller contains '<ACTOR>' or tostring(Properties) contains '<ACTOR>'
| project TimeGenerated, Caller, OperationNameValue, ActivityStatusValue, Properties


In [ ]:
# Splunk SPL
index=cloud (Operation="RoleAssignmentCreated" OR Operation="Add member to role" OR Operation="AddUserToGroup")
| table _time Operation Caller TargetUser TargetRole src_ip


In [ ]:
# Elastic (KQL/EQL)
event.category:iam AND event.action:(RoleAssignmentCreated OR AddUserToGroup)


## Containment & Response

Remove unauthorized role assignments, revoke sessions for the actor, and enforce PIM/JIT.


In [ ]:
# PowerShell
## Remove from privileged group (AD example)
Remove-ADGroupMember -Identity "Domain Admins" -Members <USER> -Confirm:$false


In [ ]:
# PowerShell
## Revoke actor tokens (AzureAD)
Revoke-AzureADUserAllRefreshToken -ObjectId <ACTOR_OBJECT_ID>
